In [6]:
# disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# safe imports
import os, sys       # arcgis comes with these
import datetime      # arcgis comes with this
import numpy as np   # arcgis comes with this
import pandas as pd  # arcgis comes with this
import matplotlib.pyplot as plt

# risk imports (non-native to arcgis)
try:
    import xarray as xr  # not in arcgis
except:
    arcpy.AddError('Python library Xarray is not installed.')
    raise

# import tools
try:
    # shared folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\shared')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared')
    import arc, satfetcher, tools  

    # module folder
    sys.path.append(r'C:\Users\262272G\Documents\GitHub\tenement-tools\modules')
    sys.path.append(r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules')
    import ensemble, cog  

except:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise

In [7]:
ds = xr.open_dataset(r'C:\Users\262272G\Desktop\test\pheno_novardim.nc')
ds = ds.compute()
ds

<xarray.Dataset>
Dimensions:      (variable: 1, x: 433, y: 445)
Coordinates:
  * y            (y) float64 -2.526e+06 -2.526e+06 ... -2.539e+06 -2.539e+06
  * x            (x) float64 -1.302e+06 -1.302e+06 ... -1.29e+06 -1.29e+06
    spatial_ref  int32 3577
  * variable     (variable) object 'veg_idx'
Data variables: (12/18)
    pos_values   (variable, y, x) float32 0.1091 0.1538 0.1705 ... 0.1318 0.1432
    pos_times    (variable, y, x) float64 59.0 59.0 59.0 ... 181.0 59.0 59.0
    mos_values   (variable, y, x) float32 0.1 0.1408 0.1577 ... 0.1212 0.1297
    vos_values   (variable, y, x) float32 0.07073 0.08695 ... 0.07063 0.07792
    vos_times    (variable, y, x) float64 365.0 365.0 365.0 ... 319.0 319.0
    bse_values   (variable, y, x) float32 0.08432 0.109 ... 0.09248 0.09842
    ...           ...
    roi_values   (variable, y, x) float32 0.0002551 0.0005179 ... 0.0008657
    rod_values   (variable, y, x) float32 9.02e-05 0.0001633 ... 0.0001697
    lios_values  (variable, y, x) float32 3.414 4.333 4.852 ... 3.799 4.087
    sios_values  (variable, y, x) float32 0.469 0.8188 0.9948 ... 0.7693 0.7968
    liot_values  (variable, y, x) float32 2.156 2.957 3.266 ... 2.495 2.692
    siot_values  (variable, y, x) float32 0.5294 0.9574 1.142 ... 0.8707 0.9001
Attributes: (12/13)
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         [ 3.00000e+01  0.00000e+00 -1.30248e+06  0.00000e+00 -...
    nodatavals:        nan
    orig_collections:  ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
    orig_bands:        ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir'...
    ...                ...
    orig_bbox:         [119.16931736 -22.95760945 119.28544083 -22.84831678]
    orig_dtype:        int16
    orig_snap_bounds:  True
    orig_cell_align:   Top-left
    orig_resample:     Nearest
    res:               30

In [8]:
# try export squeezed netcdf
ds.squeeze()


<xarray.Dataset>
Dimensions:      (x: 433, y: 445)
Coordinates:
  * y            (y) float64 -2.526e+06 -2.526e+06 ... -2.539e+06 -2.539e+06
  * x            (x) float64 -1.302e+06 -1.302e+06 ... -1.29e+06 -1.29e+06
    spatial_ref  int32 3577
    variable     <U7 'veg_idx'
Data variables: (12/18)
    pos_values   (y, x) float32 0.1091 0.1538 0.1705 ... 0.1372 0.1318 0.1432
    pos_times    (y, x) float64 59.0 59.0 59.0 196.0 ... 181.0 181.0 59.0 59.0
    mos_values   (y, x) float32 0.1 0.1408 0.1577 ... 0.1279 0.1212 0.1297
    vos_values   (y, x) float32 0.07073 0.08695 0.09233 ... 0.07063 0.07792
    vos_times    (y, x) float64 365.0 365.0 365.0 365.0 ... 319.0 319.0 319.0
    bse_values   (y, x) float32 0.08432 0.109 0.1176 ... 0.09836 0.09248 0.09842
    ...           ...
    roi_values   (y, x) float32 0.0002551 0.0005179 ... 0.0006247 0.0008657
    rod_values   (y, x) float32 9.02e-05 0.0001633 ... 0.0001568 0.0001697
    lios_values  (y, x) float32 3.414 4.333 4.852 5.248 ... 2.246 3.799 4.087
    sios_values  (y, x) float32 0.469 0.8188 0.9948 ... 0.4735 0.7693 0.7968
    liot_values  (y, x) float32 2.156 2.957 3.266 3.332 ... 2.645 2.495 2.692
    siot_values  (y, x) float32 0.5294 0.9574 1.142 ... 0.9855 0.8707 0.9001
Attributes: (12/13)
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         [ 3.00000e+01  0.00000e+00 -1.30248e+06  0.00000e+00 -...
    nodatavals:        nan
    orig_collections:  ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
    orig_bands:        ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir'...
    ...                ...
    orig_bbox:         [119.16931736 -22.95760945 119.28544083 -22.84831678]
    orig_dtype:        int16
    orig_snap_bounds:  True
    orig_cell_align:   Top-left
    orig_resample:     Nearest
    res:               30

In [5]:
ds = xr.open_dataset(r'C:\Users\262272G\Desktop\test\pheno_withvardim.nc')
ds = ds.compute()
ds

<xarray.Dataset>
Dimensions:      (variable: 1, x: 433, y: 445)
Coordinates:
  * y            (y) float64 -2.526e+06 -2.526e+06 ... -2.539e+06 -2.539e+06
  * x            (x) float64 -1.302e+06 -1.302e+06 ... -1.29e+06 -1.29e+06
    spatial_ref  int32 3577
  * variable     (variable) int32 1
Data variables: (12/18)
    pos_values   (variable, y, x) float32 0.1091 0.1538 0.1705 ... 0.1318 0.1432
    pos_times    (variable, y, x) float64 59.0 59.0 59.0 ... 181.0 59.0 59.0
    mos_values   (variable, y, x) float32 0.1 0.1408 0.1577 ... 0.1212 0.1297
    vos_values   (variable, y, x) float32 0.07073 0.08695 ... 0.07063 0.07792
    vos_times    (variable, y, x) float64 365.0 365.0 365.0 ... 319.0 319.0
    bse_values   (variable, y, x) float32 0.08432 0.109 ... 0.09248 0.09842
    ...           ...
    roi_values   (variable, y, x) float32 0.0002551 0.0005179 ... 0.0008657
    rod_values   (variable, y, x) float32 9.02e-05 0.0001633 ... 0.0001697
    lios_values  (variable, y, x) float32 3.414 4.333 4.852 ... 3.799 4.087
    sios_values  (variable, y, x) float32 0.469 0.8188 0.9948 ... 0.7693 0.7968
    liot_values  (variable, y, x) float32 2.156 2.957 3.266 ... 2.495 2.692
    siot_values  (variable, y, x) float32 0.5294 0.9574 1.142 ... 0.8707 0.9001
Attributes: (12/13)
    crs:               EPSG:3577
    grid_mapping:      spatial_ref
    transform:         [ 3.00000e+01  0.00000e+00 -1.30248e+06  0.00000e+00 -...
    nodatavals:        nan
    orig_collections:  ['ga_ls5t_ard_3', 'ga_ls7e_ard_3', 'ga_ls8c_ard_3']
    orig_bands:        ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir'...
    ...                ...
    orig_bbox:         [119.16931736 -22.95760945 119.28544083 -22.84831678]
    orig_dtype:        int16
    orig_snap_bounds:  True
    orig_cell_align:   Top-left
    orig_resample:     Nearest
    res:               30

In [21]:
# for current project, open current map
aprx = arcpy.mp.ArcGISProject('CURRENT')
m = aprx.activeMap

# create output folder using datetime as name
dt = datetime.datetime.now().strftime('%d%m%Y%H%M%S')
out_folder = os.path.join(os.path.dirname(out_nc), 'cva' + '_' + dt)
os.makedirs(out_folder)                

# disable visualise on map temporarily
arcpy.env.addOutputsToMap = False

# create crf filename and copy it
out_file = os.path.join(out_folder, 'cva.crf')
crf = arcpy.CopyRaster_management(in_raster=out_nc, 
                                  out_rasterdataset=out_file)

# add to map                  
m.addDataFromPath(crf) 

# re-enable add to map
arcpy.env.addOutputsToMap = True

# tailor cmap depending on method and parameters
#if in_trend_method == 'Mann-Kendall':
    #if mk_dir == 'both':
        #cmap = 'Red-Blue (Continuous)'
    #elif mk_dir == 'inc':
        #cmap = 'Yellow-Green-Blue (Continuous)'
    #else:
        #cmap = 'Yellow-Orange-Red (Continuous)'   
#else:
    #cmap = 'Red-Blue (Continuous)'

# apply cmap
#lyr = arc.apply_cmap(aprx=aprx, 
                     #lyr_name='trend.crf', 
                     #cmap_name=cmap, 
                     #cutoff_pct=0.0001) 

# invert if mk dec
#if in_trend_method == 'Mann-Kendall' and mk_dir == 'dec':
    #lyr.invertColorRamp = True